# Open Notebook Wrapper Script Documentation

This notebook documents the usage of the `open_notebook_wrapper.sh` script, which allows you to interact with Open Notebook from outside the Docker container without having to modify the container itself.

## Introduction

The wrapper script acts as a bridge between your host system and the Open Notebook CLI running inside the Docker container. It forwards your commands to the container and returns the results, making it seamless to interact with Open Notebook without having to connect to the container directly.

This approach has several advantages:

1. No modifications to the container are required
2. You can run commands directly from your host terminal
3. The script can be integrated with other host-side tools and scripts
4. It handles the container interaction details for you

## Setup

Before using the wrapper script, make sure to set it as executable:

```bash
chmod +x open_notebook_wrapper.sh
```

The script assumes that:

1. Docker is installed and running on your system
2. The Open Notebook container is running with the name `assistant-notebook-1`
3. The Open Notebook CLI script (`open_notebook_cli.py`) is available in the container's working directory

## Basic Usage

The wrapper script uses the same command structure as the Open Notebook CLI, but you execute it from your host system:

```bash
./open_notebook_wrapper.sh <command> [options]
```

To see the help message with all available commands:

```bash
./open_notebook_wrapper.sh --help
```

The script will check if the container is running and then forward your command to the Open Notebook CLI inside the container.

## Examples

Let's look at some examples of how to use the wrapper script:

### Notebook Management

```bash
# List all notebooks
./open_notebook_wrapper.sh list-notebooks

# Create a new notebook
./open_notebook_wrapper.sh create-notebook "Research Project" "My research notes and sources"

# Get details about a specific notebook
./open_notebook_wrapper.sh get-notebook notebook:abc123

# Archive a notebook
./open_notebook_wrapper.sh archive-notebook notebook:abc123
```

### Source Management

```bash
# List sources in a notebook
./open_notebook_wrapper.sh list-sources notebook:abc123

# Add a text source to a notebook
./open_notebook_wrapper.sh add-text-source notebook:abc123 "Important Article" "This is the content of the article..."

# Add a URL source and generate embeddings
./open_notebook_wrapper.sh add-url-source notebook:abc123 https://example.com/article --embed

# Get source details with insights
./open_notebook_wrapper.sh get-source source:abc123 --show-insights
```

### Note Management

```bash
# List notes in a notebook
./open_notebook_wrapper.sh list-notes notebook:abc123

# Create a new note
./open_notebook_wrapper.sh create-note notebook:abc123 "Meeting Notes" "Notes from today's meeting..." --type human

# Convert an insight to a note
./open_notebook_wrapper.sh insight-to-note source_insight:abc123 notebook:def456
```

### Transformations

```bash
# List all transformations
./open_notebook_wrapper.sh list-transformations

# Create a new transformation
./open_notebook_wrapper.sh create-transformation "Key Points" "Key Points" "Extracts key points from text" "Extract the 5 most important points from this text."

# Apply a transformation to a source
./open_notebook_wrapper.sh apply-transformation source:abc123 transformation:def456
```

### Search

```bash
# Text search
./open_notebook_wrapper.sh text-search "quantum computing"

# Vector (semantic) search
./open_notebook_wrapper.sh vector-search "how do quantum computers work" --results 3
```

### Podcast Generation

```bash
# List podcast templates
./open_notebook_wrapper.sh list-podcast-templates

# Generate a podcast from a notebook
./open_notebook_wrapper.sh generate-podcast podcast_config:abc123 notebook:def456 "Quantum Computing Explained"
```

## Special Commands

The wrapper script includes a special command not available in the original CLI:

### Shell Access

To get a shell inside the container (useful for debugging or more complex operations):

```bash
./open_notebook_wrapper.sh shell
```

This will open a bash shell in the container, allowing you to run commands directly.

## JSON Output

For automated processing or integration with other tools, you can request JSON output by adding the `--json` flag:

```bash
./open_notebook_wrapper.sh list-notebooks --json
./open_notebook_wrapper.sh get-notebook notebook:abc123 --json
```

This outputs data in structured JSON format, which can be parsed by other tools or scripts.

## Error Handling

The wrapper script includes basic error handling:

1. It checks if the container is running before attempting to execute commands
2. It forwards any error messages from the CLI inside the container
3. It exits with an error status if the container is not available

Example of a common error:

```
Error: Container 'assistant-notebook-1' is not running
Please make sure the Open Notebook Docker container is started
```

## Integration with Other Scripts

The wrapper script can be easily integrated with other shell scripts or tools on your host system. Here's an example of a simple automation script that creates a notebook and adds several URL sources to it:

In [ ]:
#!/bin/bash
# Example: Creating a research notebook with multiple sources

# Create a new notebook and capture its ID from the JSON output
NOTEBOOK_ID=$(./open_notebook_wrapper.sh create-notebook "AI Research" "Collection of AI research papers" --json | jq -r '.id')
echo "Created notebook: $NOTEBOOK_ID"

# List of URLs to add as sources
URLS=(
  "https://arxiv.org/abs/1706.03762"
  "https://arxiv.org/abs/1810.04805"
  "https://arxiv.org/abs/2005.14165"
)

# Add each URL as a source to the notebook
for URL in "${URLS[@]}"; do
  echo "Adding source: $URL"
  ./open_notebook_wrapper.sh add-url-source "$NOTEBOOK_ID" "$URL" --embed
done

# List the sources in the notebook
echo "Sources in the notebook:"
./open_notebook_wrapper.sh list-sources "$NOTEBOOK_ID"

## Customizing the Wrapper

You can customize the wrapper script to fit your specific environment by modifying the following variables at the top of the script:

- `CONTAINER_NAME`: If your Open Notebook container has a different name
- `CLI_PATH`: If the CLI script is located in a different directory in the container

For example, if your container is named "open-notebook" and the CLI is in a different path:

```bash
CONTAINER_NAME="open-notebook"
CLI_PATH="/app/open_notebook_cli.py"
```

## Conclusion

The Open Notebook wrapper script provides a convenient way to interact with Open Notebook from your host system without having to modify the container. It supports all the functionality of the Open Notebook CLI while handling the container interaction details for you.

By using this wrapper, you can automate Open Notebook operations, integrate with other tools, and build more complex workflows around your Open Notebook instance.